In [2]:
import os
os.environ.setdefault('AWS_PROFILE', 'ghactivity')


'ghactivity'

In [3]:
!aws s3 rm  s3://sgghactivity/ --recursive

In [3]:
import boto3
dynamodb_table=boto3.resource('dynamodb')
table= dynamodb_table.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': 'D8AMSNDBG03BOEHU160V3MIRN7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 25 Jul 2022 15:13:44 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D8AMSNDBG03BOEHU160V3MIRN7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [17]:
def get_job_details(job_name):
    dynamodb=boto3.resource('dynamodb')
    table= dynamodb_table.Table('jobs')
    job_details=table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [18]:
get_job_details('ghactivity_ingest')

{'job_description': 'ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [19]:
from datetime import datetime as dt
from datetime import timedelta as td
import time

In [21]:
def get_next_file_name(job_details):
    job_start_time=int(time.mktime(dt.now().timetuple()))
    job_run_bookmark_details=job_details.get('job_run_bookmark_details')
    baseline_days = int(job_details['baseline_days'])
    if job_run_bookmark_details:
        dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0].split('/')[-1]
        next_file_name = f"{dt.strftime(dt.strptime(dt_part,'%Y-%m-%d-%H')+td(hours=1), '%Y-%m-%d-%-H')}.json.gz"
    else:
        next_file_name = f'{dt.strftime(dt.now().date()-td(days=3), "%Y-%m-%d")}-0.json.gz'
    return job_start_time,next_file_name
        

In [24]:
job_details = get_job_details('ghactivity_ingest')

In [26]:
job_start_time, next_file = get_next_file_name(job_details)

In [27]:
next_file

'2022-07-22-0.json.gz'

In [28]:
job_start_time

1658762268

In [29]:
import requests

In [30]:
bucket_name = 'sgghactivity'

In [31]:
folder = 'landing'

In [32]:
def upload_file_to_s3(file_name, bucket_name, folder):
    print(f'getting the {file_name} from gharchive')
    res= requests.get(f'https://data.gharchive.org/{file_name}')
    print(f' uploading {file_name} to s3 under s3://{bucket_name}/{folder}')
    s3_client=boto3.client('s3')
    upload_res=s3_client.put_object(
        Bucket=bucket_name,
        Key=f'{folder}/{file_name}',
        Body=res.content
    )

    return{
        'last_run_file_name': f's3://{bucket_name}/{folder}/{file_name}',
        'status_code': upload_res['ResponseMetadata']['HTTPStatusCode']

    }


In [33]:
jobs_run_details=upload_file_to_s3(next_file, bucket_name, folder)

getting the 2022-07-22-0.json.gz from gharchive
 uploading 2022-07-22-0.json.gz to s3 under s3://sgghactivity/landing


In [34]:
def save_job_run_details(job_details, jobs_run_details, job_start_time):
    dynamodb= boto3.resource('dynamodb')
    jobs_run_details_item = {
        'job_id': job_details['job_id'],
        'jobs_run_time': job_start_time,
        'job_run_bookmark_details': jobs_run_details,
        'create_ts': int(time.mktime(dt.now().timetuple()))
    }
    jobs_run_details_table = dynamodb.Table('jobs_run_details')
    jobs_run_details_table.put_item(Item=jobs_run_details_item)
    job_details_table=dynamodb.Table('jobs')
    job_details['job_run_bookmark_details']=jobs_run_details
    job_details_table.put_item(Item=job_details)

In [57]:
save_job_run_details(job_details, jobs_run_details, job_start_time)

In [59]:
!aws s3 ls s3://sgghactivity/landing --recursive

2022-07-21 21:00:14        127 landing/2022-07-18-0.json
